In [1]:
#Bibliotecas
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pandas as pd

In [2]:
#Definindo arquivos e Variáveis

#path = ('C:\\Users\\Ana\\Desktop\\Desafio_Engenharia_Dados\\Atividade 01\\')

contrato = ['contrato.csv']
transacao = ['transacao.csv']
cliente = ['cliente.csv']

spark = SparkSession \
    .builder \
    .appName ("read csv file") \
    .getOrCreate ()

In [3]:
#Lendo Arquivos CSV
df_contrato = spark.read.option('header', 'true').option('sep', ';').csv(contrato)
df_transacao = spark.read.option('header', 'true').option('sep', ';').csv(transacao)
df_cliente = spark.read.option('header', 'true').option('sep', ';').csv(cliente)

In [4]:
#Tratando campos nulos e juntando colunas das tabelas
df_tables = df_contrato.join(df_transacao, on='contrato_id', how='inner')
df_tables = df_tables.join(df_cliente, on='cliente_id', how='inner')
df_tables = df_tables.withColumn("percentual_desconto", \
F.when(F.col("percentual_desconto")=="null" ,0)\
                                    .otherwise(F.col("percentual_desconto")))

In [5]:
#Valor total com percentual desconto
desc_percentual = (F.col('valor_total')) - (F.col('valor_total') * (F.col('percentual_desconto')/100))

In [6]:
df_tables = df_tables.withColumn('valor_liquido', desc_percentual)

In [7]:
#Filtrando informações
df_soma = df_tables.filter(df_tables.ativo == 1)\
                        .filter(df_tables.valor_total> 1)

In [8]:
df_total = df_soma.select(F.col("Valor_liquido")).groupBy().sum().withColumnRenamed("sum(valor_liquido)","valor")

In [9]:
#Tratando valores da coluna 'valor'
df_total = df_total.select(F.round(F.col("valor"), 2)).withColumnRenamed("round(valor, 2)","valor")

In [10]:
df_total.show()

+--------+
|   valor|
+--------+
|59973.46|
+--------+

